In [83]:
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
import numpy as np
output = """
8.4e-2 in**3=PROGRAM:=```python\n# Q0: If I were to take an English penny, melt it down, then somehow stretch it out into a perfect hollow sphere 1 atom thick, how large would this sphere be?\n# Q1: What is the volume of a single penny?\nA1=0.93 # volume of a single penny (in**3)\n# Q2: What is the surface area of a hollow sphere 1 atom thick which is constructed from a melted penny?\nA2=0.93 # surface area of a hollow 1 atom thick sphere constructed from a melted penny (in**2)\nA0=(4/3)*np.pi*(A2/(4*np.pi))**(3/2) # volume of a hollow 1 atom thick sphere constructed from a melted penny (in**3)```
"""
program = output[output.find("```python")+len("```python"):output.rfind("```")]
exec(program)
A0

0.08433313325423158

In [25]:
(16 * 209)/0.012

278666.6666666667

In [148]:
raw_example1 = {
    "question": "How many air molecules are in this lecture hall?",
    "program": "PROGRAM:=Q1: What is the volume of the lecture hall?=Q2: What is the volume of a single air molecule?=A1: 24000 m**3=A2: 3e-24 m**3=Q2 -> A2 | F2=Q1 -> A1 | F1=P: Div (Q1, Q2)",
    "answer": "8.00E+27",
    "context": "CONTEXT:=F1: The volume of the lecture hall is 24000 metre cube=F2: The volume of a single air molecule is 3e-24 cubic meter"
}

ideal_variables = """
'lecture_hall_volume': 'What is the volume of the lecture hall?'
'air_molecule_volume': 'What is the volume of a single air molecule?'
"""

variables = """
'lecture_hall_volume': 24000
'air_molecule_volume': 3e-24
"""

expression = """
```lecture_hall_volume / air_molecule_volume```
"""

example1 = {
    **raw_example1,
    "context": raw_example1["context"][len("CONTEXT:"):].replace("=", "\n"),
    "ideal_variables": ideal_variables,
    "variables": variables,
    "expression": expression
}


In [149]:
raw_example2 = {
    "question": "How long would it take to pluck each hair out of your body one at a time?",
    "program": "PROGRAM:=Q1: How long does it take to pluck a single strand of hair?=Q2: How many hairs do we have on our body?=A1: 5 s=A2: 5e+6=Q2 -> A2 | F2=Q1 -> A1 | F1=P: Mul (Q1, Q2)",
    "answer": "2.5e+7 s",
    "context": "CONTEXT:=F1: It takes around 5 seconds to pluck a single strand of hair.=F2: The entire human body has 5e+6 hair follicles."
}


ideal_variables = """
'number_body_hairs': 'How many hairs do we have on our body?'
'time_to_pluck_hair': 'How long does it take to pluck a single strand of hair?'
"""

variables = """
'number_body_hairs': 5e+6
'time_to_pluck_hair': 5
"""

expression = """
```number_body_hairs * time_to_pluck_hair```
"""

example2 = {
    **raw_example2,
    "context": raw_example2["context"][len("CONTEXT:"):].replace("=", "\n"),
    "ideal_variables": ideal_variables,
    "variables": variables,
    "expression": expression
}

In [150]:
raw_example3 = {
        "question": "How many golf balls put into the worlds oceans would it take to submerge all of the land on earth from the displaced water?",
        "program": "PROGRAM:=Q1: What is the volume of the oceans?=Q2: What is the volume of a golf ball?=A1: 1.3e+9 km**3=A2: 4e-5 km**3=Q2 -> A2 | F10=Q1 -> A1 | F11=P: Div (Q1, Q2)",
        "answer": "3.20E+13",
        "context": "CONTEXT:=F1: A single nuke contains around 4e+15 J=F2: Earth is to be moved 1% out of its orbit.=F3: The gravitational constant of earth is 6.6e-11 m**3*kg**-1*s**-2=F4: The volume of single SO bacterium is 2e-12 cc=F5: The surface area of oceans is 361e+6 km**2=F6: The average volume of a living thing in an ocean is 5e-3 km**3=F7: The radius of earth is 6.3e+6 m=F8: The average weight of normal person is 63 kg=F9: The height of the water in ocean is 3.7 km=F10: The volume of a golf ball is 4e-5 km**3=F11: The volume of the oceans is 1.3e+9 km**3=F12: The number of living things in an ocean is 1e+9=F13: The weight of a megalodon is 66138 pounds.=F14: Even Though dead bodies float, dead people weighing upwards of 125 kg sink in water.=F15: The numerical coefficient to calculate the square of the radius is 2.=F16: The towing capacity of a Model X is 6700 pounds=F17: The original volume of the oceans is 1335700000 km**3=F18: Around 4e+35 J is required to knock earth from its orbit.=F19: The mass of earth is 6e+24 kg=F20: The volume of ping pong ball is 33.5 cc",
        "fact_transform": {
            "10": 0,
            "9": 1
        }
    }
example3 = {
    **raw_example3,
    "context": raw_example3["context"][len("CONTEXT:"):].replace("=", "\n"),
}

In [151]:
from langchain.llms.openai import OpenAI
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.chains import LLMChain

In [185]:
SYSTEM_PROMPT_STRING = """
    You are a helpful assistant tasked with answering estimation questions using provided information.
    You are given a question
"""


INPUT_PROMPT = PromptTemplate(
                        input_variables=["question", "context"],
                        template="""Question: {question}"""
                    )

OUTPUT_PROMPT = PromptTemplate(
                        input_variables=["ideal_variables", "expression"],
                        template="""Desired Quantites:{ideal_variables}\nExpression:{expression}"""
                    )

EXAMPLE_PROMPT = PromptTemplate.from_template(
                        """INPUT:\n{input}\n\nOUTPUT:\n{output}"""
                    )


examples = [
    {
        "input": INPUT_PROMPT.format(question=ex['question']),
        "output": OUTPUT_PROMPT.format(ideal_variables=ex['ideal_variables'], expression=ex['expression']),
    }
    for ex in [example1, example2]
]

PROMPT = FewShotPromptTemplate(
    examples=examples,
    example_prompt=EXAMPLE_PROMPT,
    suffix="INPUT:\n{input}",
    input_variables=["input"]
)



In [186]:
llm = OpenAI(temperature=0.0,)
llm_chain = LLMChain(llm=llm, prompt=PROMPT)


In [193]:
question = """What would be the effect to the oceans If all fishing was banned for four days once a year across the planet?"""

response = llm_chain.invoke(input=INPUT_PROMPT.format(question=question))
print(response["text"])



OUTPUT:
Desired Quantites:
'fishing_frequency': 'How often is fishing done in the oceans?'
'fishing_duration': 'How long does it take to fish in the oceans?'

Expression:
```fishing_frequency * fishing_duration * 4```


In [113]:
import json

vars = json.loads(variables)

y = eval(program, vars)


8e+27


In [3]:
def input_extraction(datapoint):
    extracted_input = {
        "question": datapoint['question'],
        "context": datapoint['context'][len("CONTEXT:"):].replace('=', '\n'),
    }
    return extracted_input

In [62]:
SYSTEM_PROMPT_STRING = """
    You are a helpful assistant tasked with answering estimation questions using provided information.
    You are provided with contextual information which contains facts that is relevant to answering the estimation question.
    You should output a mathematical formula to calculate the answer but do not evaluate the formula.
"""

INPUT_PROMPT = PromptTemplate(
                        input_variables=["context", "question"],
                        template="""CONTEXT:{context}\n\nQUESTION: {question}"""
                    )

REASONER_PROMPT = PromptTemplate(
    template=f"{SYSTEM_PROMPT_STRING}\n\n{INPUT_PROMPT.template}",
    input_variables=INPUT_PROMPT.input_variables,
)



In [64]:
llm = OpenAI(temperature=0)
# reasoner_chain = LLMChain(llm=llm, verbose=True)
math_chain = LLMMathChain(llm=llm, verbose=True)
chain = math_chain


/Users/sidhartkrishnan/Documents/Programming/Projects/cs236/carafe/.venv/lib/python3.11/site-packages/langchain/chains/llm_math/base.py:57: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [53]:
raw_example1 = {
    "question": "How many air molecules are in this lecture hall?",
    "program": "PROGRAM:=Q1: What is the volume of the lecture hall?=Q2: What is the volume of a single air molecule?=A1: 24000 m**3=A2: 3e-24 m**3=Q2 -> A2 | F2=Q1 -> A1 | F1=P: Div (Q1, Q2)",
    "answer": "8.00E+27",
    "context": "CONTEXT:=F1: The volume of the lecture hall is 24000 metre cube=F2: The volume of a single air molecule is 3e-24 cubic meter"
}

raw_example2 = {
    "question": "How long would it take to pluck each hair out of your body one at a time?",
    "program": "PROGRAM:=Q1: How long does it take to pluck a single strand of hair?=Q2: How many hairs do we have on our body?=A1: 5 s=A2: 5e+6=Q2 -> A2 | F2=Q1 -> A1 | F1=P: Mul (Q1, Q2)",
    "answer": "2.5e+7 s",
    "context": "CONTEXT:=F1: It takes around 5 seconds to pluck a single strand of hair.=F2: The entire human body has 5e+6 hair follicles."
}

raw_example3 = {
        "question": "How many golf balls put into the worlds oceans would it take to submerge all of the land on earth from the displaced water?",
        "program": "PROGRAM:=Q1: What is the volume of the oceans?=Q2: What is the volume of a golf ball?=A1: 1.3e+9 km**3=A2: 4e-5 km**3=Q2 -> A2 | F10=Q1 -> A1 | F11=P: Div (Q1, Q2)",
        "answer": "3.20E+13",
        "context": "CONTEXT:=F1: A single nuke contains around 4e+15 J=F2: Earth is to be moved 1% out of its orbit.=F3: The gravitational constant of earth is 6.6e-11 m**3*kg**-1*s**-2=F4: The volume of single SO bacterium is 2e-12 cc=F5: The surface area of oceans is 361e+6 km**2=F6: The average volume of a living thing in an ocean is 5e-3 km**3=F7: The radius of earth is 6.3e+6 m=F8: The average weight of normal person is 63 kg=F9: The height of the water in ocean is 3.7 km=F10: The volume of a golf ball is 4e-5 km**3=F11: The volume of the oceans is 1.3e+9 km**3=F12: The number of living things in an ocean is 1e+9=F13: The weight of a megalodon is 66138 pounds.=F14: Even Though dead bodies float, dead people weighing upwards of 125 kg sink in water.=F15: The numerical coefficient to calculate the square of the radius is 2.=F16: The towing capacity of a Model X is 6700 pounds=F17: The original volume of the oceans is 1335700000 km**3=F18: Around 4e+35 J is required to knock earth from its orbit.=F19: The mass of earth is 6e+24 kg=F20: The volume of ping pong ball is 33.5 cc",
        "fact_transform": {
            "10": 0,
            "9": 1
        }
    }



In [55]:
from langchain.tools import Tool, StructuredTool
from langchain.chains import LLMMathChain

llm = OpenAI(temperature=0)
llm_math_chain = LLMMathChain(llm=llm, verbose=True)

math = StructuredTool.from_function(
    func=llm_math_chain.run,
    name="Math",
    description="Perform mathematical calculations"
)

/Users/sidhartkrishnan/Documents/Programming/Projects/cs236/carafe/.venv/lib/python3.11/site-packages/langchain/chains/llm_math/base.py:57: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [81]:
from langchain.agents import initialize_agent
from langchain.tools import Tool



math_chain = LLMMathChain(llm=llm, verbose=True)

reason_tool = Tool(name="Reason", func=llm.__call__, description="Reason about the question")
math_tool = Tool(name="Calculate", func=math_chain.run, description="Calculate the answer")

tools = [reason_tool, math_tool]

agent = initialize_agent(tools, llm)

def run(input):
    formula = agent.run(REASONER_PROMPT.format(**input_extraction(input)))
    # return agent.run("Calculate: " + formula)

/Users/sidhartkrishnan/Documents/Programming/Projects/cs236/carafe/.venv/lib/python3.11/site-packages/langchain/chains/llm_math/base.py:57: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [82]:
run(raw_example3)



> Entering new LLMMathChain chain...
Volume of displaced water
```text
(4/3)*pi*(2.5**3)
```
...numexpr.evaluate("(4/3)*pi*(2.5**3)")...

Answer: 65.44984694978736
> Finished chain.


> Entering new LLMMathChain chain...
Volume of a single golf ball
```text
(4/3)*pi*(1.68/2)**3
```
...numexpr.evaluate("(4/3)*pi*(1.68/2)**3")...

Answer: 2.4827127095377124
> Finished chain.


> Entering new LLMMathChain chain...
Total volume of golf balls needed
This question cannot be answered using the numexpr library.

ValueError: unknown format from LLM: This question cannot be answered using the numexpr library.